<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 7:** Retrieval-Augmented Generation with Vector Stores</font>

<br>

In the previous notebook, we learned about embedding models and exercised some of their capabilities. We discussed their intended use cases of longer-form document comparison and found ways to use it as a backbone for more custom semantic comparisons. This notebook will progress these ideas toward the retrieval model's intended use case and explore how to build chatbot systems that rely on *vector stores* to automatically save and retrieve information.

<br>

### **Learning Objectives:**

- Understand how semantic-similarity-backed systems can facilitate easy-to-use retrieval formulations.

- Learn how to incorporate retrieval modules into your chat model systems for a retrieval-augmented generation (RAG) pipeline, which can be applied to tasks like document retrieval and conversation memory buffers.

<br>

### **Questions To Think About:**

- This notebook does not attempt to incorporate hierarchical reasoning or non-naive RAG (such as planning agents). Consider what modifications would be necessary to make these components work in an LCEL chain.

- Consider when it would be best to move your vector store solution into a scalable service and when a GPU will become necessary for optimization.

<br>

### **Environment Setup:**

In [1]:
# %%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

----

<br>

## Part 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/)

----

<br>

## **Part 2:** RAG for Conversation History

In our previous explorations, we delved into the capabilities of document embedding models and used them to embed, store, and compare semantic vector representations of text. Though we could motivate how to efficiently extend this into vector store land manually, the true beauty of working with a standard API is its strong incorporation with other frameworks that can already do the heavy lifting for us!

<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [3]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>

### **Step 2:** Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)

<br>

In addition to simplifying the process from an API perspective, vector stores also implement connectors, integrations, and optimizations under the hood. In our case, we will start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine!

**Specifically:**

1. We can feed our conversation into [**a FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss) via the `from_texts` constructor. This will take our conversational data and the embedding model to create a searchable index over our discussion.
2. This vector store can then be "interpreted" as a retriever, supporting the LangChain runnable API and returning documents retrieved via an input query.

The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [4]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 243 ms, sys: 365 ms, total: 608 ms
Wall time: 1.81 s


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [5]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        id='6eeb0d13-e80b-4cd6-9bd1-6d00514d34eb',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='aef342a2-16a4-4da5-bb7c-1bb38d6098f9',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        id='f4286431-b896-4283-8260-43f7398bd56d',
        metadata={},
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        id='fe3c3a4a-8f62-447a-8f92-f9ea5365c365',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [6]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        id='78cee2ff-1f6e-4dc8-bdf7-102d732ef3d4',
        metadata={},
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        id='fe3c3a4a-8f62-447a-8f92-f9ea5365c365',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        id='6eeb0d13-e80b-4cd6-9bd1-6d00514d34eb',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='aef342a2-16a4-4da5-bb7c-1bb38d6098f9',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

As we can see, our retriever found a handful of semantically relevant documents from our query. You may notice that not all of the documents are useful or clear on their own. For example, a retrieval of *"Beras"* for *"your name"* may be problematic for the chatbot if provided out of context. Anticipating the potential problems and creating synergies between your LLM components can increase the likelihood of good RAG behavior, so keep an eye out for such pitfalls and opportunities.

<br>

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [7]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [8]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

 Based on the conversation, Beras mentioned that they live in the Arctic.

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [9]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

 Hello! The Rocky Mountains stretch across North America. They are a beautiful and majestic range of mountains that
span across this region.

In [10]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

 Hello there! The Rocky Mountains are indeed a magnificent range of mountains that stretch across North America. 
They're not immediately close to California, though. They actually span from British Columbia in Canada, through 
the United States, and down to New Mexico. So, while they're not right next to California, you might be able to see
them if you travel far enough north or east from the state.

In [11]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

 Although I can't provide specific details about the exact location of Beras, I can tell you that the Rocky 
Mountains are a large mountain range in North America. However, without knowing Beras's precise location in the 
Arctic, it's not possible for me to determine the distance between Beras and the Rocky Mountains.

<br>

You might notice some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

 While I'm sure the Rocky Mountains are an excellent place to enjoy some ice cream, our conversation was mainly 
about the mountains themselves! The Rocky Mountains are known for their stunning beauty and geological 
significance. They stretch across North America, providing breathtaking views and unique ecological systems. If you
have any questions or would like to know more about them, I'd be happy to share!

 Based on our conversation about the Rocky Mountains, it seems like you have a fondness for ice cream! Although 
it's not directly related to the mountains, it certainly sounds like a sweet treat to enjoy while taking in the 
views. So, I'd guess that ice cream might just be one of your favorite foods!

 Oh, I see! Well, it seems like my guess was a bit off. It's fascinating how we all have our unique preferences, 
isn't it? Your fondness for honey certainly adds a sweet touch to your personality, Beras. Next time, I'll do my 
best to make a more accurate guess about your favorite food. The Rocky Mountains are still a fantastic topic, 
though. They offer a variety of sights and activities, and I'm sure they could pair well with your favorite 
honey-based treats. How do you usually enjoy your honey, by the way?

 Well, I remember our conversation quite well! At first, I did think that ice cream was your favorite food, but 
then you kindly corrected me and revealed that honey is your top pick. So, yes, I do know your favorite food now - 
honey! Could you tell me more about how you usually enjoy your honey? I'm curious!

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## **Part 3 [Exercise]:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!

<br>

### **Exercise:**

In the previous example, you may recall that we pulled in some relatively small papers with the help of [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) using the following syntax:

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

Given all that you've learned so far, choose a selection of papers that you would like to use and develop a chatbot that can talk about them!

<br>

Though this is a pretty big task, a walkthrough of ***most*** of the process will be provided below. By the end of the walkthrough, many of the necessary puzzle pieces will be provided, and your real task will be to integrate them together for the final `retrieval_chain`. When you're done, get ready to re-integrate the chain (or a flavor of your choice) in the last notebook as part of the evaluation exercise!


<br>

### **Task 1**: Loading And Chunking Your Documents

The following code block gives you some default papers to load in for your RAG chain. Feel free to select more papers as desired, but note that longer documents will take longer to process. A few simplifying assumptions and additional processing steps are included to help you improve your naive RAG performance:

- Documents are cut off prior to the "References" section if one exists. This will keep our system from considering the citations and appendix sections, which tend to be long and distracting.

- A chunk that lists the available documents is inserted to provide a high-level view of all available documents in a single chunk. If your pipeline does not provide metadata on each retrieval, this is a useful component and can even be listed among a list of higher-priority pieces if appropriate.

- Additionally, the metadata entries are also inserted to provide general information. Ideally, there would also be some synthetic chunks that merge the metadata into interesting cross-document chunks.

**NOTE:** ***For the sake of the assessment, please include at least one paper that is less than one month old!***


In [13]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader
from langchain.retrievers import ArxivRetriever

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)    

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [doc[0] for doc in ArxivRetriever().batch([
    "1706.03762",     ## Attention Is All You Need Paper
    "1810.04805",     ## BERT Paper
    "2005.11401",     ## RAG Paper
    "2310.06825",     ## Mistral Paper
    "2306.05685",     ## LLM-as-a-Judge
    "2504.20734"      ## Universal RAG
    ## Some longer papers
    # "2210.03629",   ## ReAct Paper
    # "2112.10752",   ## Latent Stable Diffusion Paper
    # "2103.00020",   ## CLIP Paper
    ## TODO: Feel free to add more
])] + ArxivLoader(query="Retrieval-Augmented Generation", load_max_docs=2).load()

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc.page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents([doc]) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena
 - UniversalRAG: Retrieval-Augmented Generation over Corpora of Diverse Modalities and Granularities
 - FFPDG: Fast, Fair and Private Data Generation
 - Toward Co-creative Dungeon Generation via Transfer Learning
 - TaikoNation: Patterning-focused Chart Generation for Rhythm Action Games 

Document 0
 - # Chunks: 2
 - Metadata: 


{
    'Entry ID': 'http://arxiv.org/abs/1706.03762v7',
    'Published': datetime.date(2023, 8, 2),
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin'
}


Document 1
 - # Chunks: 2
 - Metadata: 


...

{
    'Published': '2023-06-30',
    'Title': 'FFPDG: Fast, Fair and Private Data Generation',
    'Authors': 'Weijie Xu, Jinjin Zhao, Francis Iannacci, Bo Wang',
    'Summary': 'Generative modeling has been used frequently in synthetic data generation. Fairness and privacy are
two big concerns for synthetic data. Although Recent GAN [\\cite{goodfellow2014generative}] based methods show good
results in preserving privacy, the generated data may be more biased. At the same time, these methods require high 
computation resources. In this work, we design a fast, fair, flexible and private data generation method. We show 
the effectiveness of our method theoretically and empirically. We show that models trained on data generated by the
proposed method can perform well (in inference stage) on real application scenarios.'
}


Document 7
 - # Chunks: 58
 - Metadata: 


{
    'Published': '2021-07-27',
    'Title': 'Toward Co-creative Dungeon Generation via Transfer Learning',
    'Authors': 'Zisen Zhou, Matthew Guzdial',
    'Summary': 'Co-creative Procedural Content Generation via Machine Learning (PCGML) refers to systems where a 
PCGML agent and a human work together to produce output content. One of the limitations of co-creative PCGML is 
that it requires co-creative training data for a PCGML agent to learn to interact with humans. However, acquiring 
this data is a difficult and time-consuming process. In this work, we propose approximating human-AI interaction 
data and employing transfer learning to adapt learned co-creative knowledge from one game to a different game. We 
explore this approach for co-creative Zelda dungeon room generation.'
}


Document 8
 - # Chunks: 59
 - Metadata: 


{
    'Published': '2021-07-26',
    'Title': 'TaikoNation: Patterning-focused Chart Generation for Rhythm Action Games',
    'Authors': 'Emily Halina, Matthew Guzdial',
    'Summary': 'Generating rhythm game charts from songs via machine learning has been a problem of increasing 
interest in recent years. However, all existing systems struggle to replicate human-like patterning: the placement 
of game objects in relation to each other to form congruent patterns based on events in the song. Patterning is a 
key identifier of high quality rhythm game content, seen as a necessary component in human rankings. We establish a
new approach for chart generation that produces charts with more congruent, human-like patterning than seen in 
prior work.'
}

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [14]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 423 ms, sys: 18.5 ms, total: 441 ms
Wall time: 16.3 s


<br>

From there, we can combine our indices into a single one using the following utility:

In [15]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 181 chunks


<br>

### **Task 3: [Exercise]** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [16]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!

retrieval_chain = (
    {"input": (lambda x: x)}
    ## Retrieve both:
    ##  - history from convstore (conversation memory)
    ##  - context from docstore (paper chunks)
    | RunnableAssign({
        "history": (
            itemgetter("input")
            | convstore.as_retriever()
            | long_reorder
            | partial(docs2str, title="Conversation")
        )
    })
    | RunnableAssign({
        "context": (
            itemgetter("input")
            | docstore.as_retriever()
            | long_reorder
            | partial(docs2str, title="Document")
        )
    })
)

## END TODO
################################################################################################

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about RAG!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about RAG!\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks] . We introduce RAG models where the parametric memory is a pre-trained seq2seq model
and the non-parametric memory is a dense vector index of Wikipedia, accessed with a pre-trained neural retriever. 
We compare two RAG formulations, one which conditions on the same retrieved passages across the whole generated 
sequence, the other can use different passages per token. We fine-tune and evaluate our models on a wide range of 
knowledge-intensive NLP tasks and set the state-of-the-art on three open domain QA tasks, outperforming parametric 
seq2seq models and task-specific retrieve-and-extract architectures. For language generation tasks, we find that 
RAG models generate more specific, diverse and factual language than a state-of-the-art parametric-only seq2seq 
baseline.\n[Quote from UniversalRAG: Retrieval-Augmented Generation over Corpora of Diverse Modalities and 
Granularities] . Specifically, motivated by the observation that forcing all modalities into a unified 
representation space derived from a single aggregated corpus causes a modality gap, where the retrieval tends to 
favor items from the same modality as the query, we propose modality-aware routing, which dynamically identifies 
the most appropriate modality-specific corpus and performs targeted retrieval within it, and further justify its 
effectiveness with a theoretical analysis. Moreover, beyond modality, we organize each modality into multiple 
granularity levels, enabling fine-tuned retrieval tailored to the complexity and scope of the query. We validate 
UniversalRAG on 10 benchmarks of multiple modalities, showing its superiority over various modality-specific and 
unified baselines.\n[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] Large pre-trained
language models have been shown to store factual knowledge in their parameters, and achieve state-of-the-art 
results when fine-tuned on downstream NLP tasks. However, their ability to access and precisely manipulate 
knowledge is still limited, and hence on knowledge-intensive tasks, their performance lags behind task-specific 
architectures. Additionally, providing provenance for their decisions and updating their world knowledge remain 
open research problems. Pre-trained models with a differentiable access mechanism to explicit non-parametric memory
can overcome this issue, but have so far been only investigated for extractive downstream tasks. We explore a 
general-purpose fine-tuning recipe for retrieval-augmented generation (RAG) -- models which combine pre-trained 
parametric and non-parametric memory for language generation\n[Quote from UniversalRAG: Retrieval-Augmented 
Generation over Corpora of Diverse Modalities and Granularities] Retrieval-Augmented Generation (RAG) has shown 
substantial promise in improving factual accuracy by grounding model responses with external knowledge relevant to 
queries. However, most existing approaches are limited to a text-only corpus, and while recent efforts have 
extended RAG to other modalities such as images and videos, they typically operate over a single modality-specific 
corpus. In contrast, real-world queries vary widely in the type of knowledge they require, which a single type of 
knowledge source cannot address. To address this, we introduce UniversalRAG, designed to retrieve and integrate 
knowledge from heterogeneous sources with diverse modalities and granularities\n\n\n (Answer only from retrieval. 
Only cite sources that are used. Make your response conversational.)',
            additional_kwargs={},
           

 RAG, or Retrieval-Augmented Generation, is a concept in the field of natural language processing (NLP) that involves using both parametric and non-parametric memory to improve the performance of language generation models on knowledge-intensive tasks.

Parametric memory refers to the knowledge that a model learns during training and stores in its parameters. Non-parametric memory, on the other hand, is external knowledge that a model can retrieve as needed. In the case of RAG, this non-parametric memory is accessed through a dense vector index of a large corpus, such as Wikipedia, using a pre-trained neural retriever.

There are different formulations of RAG, such as one where the same retrieved passages are used across the entire generated sequence, and another where different passages can be used for each token. RAG models have been shown to outperform parametric-only seq2seq models and task-specific retrieve-and-extract architectures on a range of knowledge-intensive NLP tasks.

In

### **Task 4:** Interact With Your Gradio Chatbot

In [25]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://094eaf58be01da97d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ChatPromptValue(
    messages=[
        SystemMessage(
            content="You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: What Modalities does Universal RAG make use of?\n\n From this, we have retrieved the following 
potentially-useful info:  Conversation History Retrieval:\n[Quote from Conversation] User previously responded with
Tell me about RAG!\n[Quote from Conversation] Agent previously responded with  RAG, or Retrieval-Augmented 
Generation, is a concept in the field of natural language processing (NLP) that involves using both parametric and 
non-parametric memory to improve the performance of language generation models on knowledge-intensive 
tasks.\n\nParametric memory refers to the knowledge that a model learns during training and stores in its 
parameters. Non-parametric memory, on the other hand, is external knowledge that a model can retrieve as needed. In
the case of RAG, this non-parametric memory is accessed through a dense vector index of a large corpus, such as 
Wikipedia, using a pre-trained neural retriever.\n\nThere are different formulations of RAG, such as one where the 
same retrieved passages are used across the entire generated sequence, and another where different passages can be 
used for each token. RAG models have been shown to outperform parametric-only seq2seq models and task-specific 
retrieve-and-extract architectures on a range of knowledge-intensive NLP tasks.\n\nIn addition to the original RAG 
model, there is also a variant called UniversalRAG, which is designed to retrieve and integrate knowledge from 
heterogeneous sources with diverse modalities and granularities. This is in contrast to most existing approaches, 
which are limited to a text-only corpus. UniversalRAG includes a modality-aware routing mechanism, which 
dynamically identifies the most appropriate modality-specific corpus and performs targeted retrieval within it. It 
has been shown to outperform various modality-specific and unified baselines on 10 benchmarks of multiple 
modalities.\n\nOverall, RAG and UniversalRAG offer a promising approach to improving the factual accuracy and 
performance of language generation models on knowledge-intensive tasks by allowing them to access and precisely 
manipulate external knowledge as needed.\n\n\n Document Retrieval:\n[Quote from UniversalRAG: Retrieval-Augmented 
Generation over Corpora of Diverse Modalities and Granularities] . Specifically, motivated by the observation that 
forcing all modalities into a unified representation space derived from a single aggregated corpus causes a 
modality gap, where the retrieval tends to favor items from the same modality as the query, we propose 
modality-aware routing, which dynamically identifies the most appropriate modality-specific corpus and performs 
targeted retrieval within it, and further justify its effectiveness with a theoretical analysis. Moreover, beyond 
modality, we organize each modality into multiple granularity levels, enabling fine-tuned retrieval tailored to the
complexity and scope of the query. We validate UniversalRAG on 10 benchmarks of multiple modalities, showing its 
superiority over various modality-specific and unified baselines.\n[Quote from Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks] . We introduce RAG models where the parametric memory is a pre-trained seq2seq model
and the non-parametric memory is a dense vector index of Wikipedia, accessed with a pre-trained neural retriever. 
We compare two RAG formulations, one which conditions on the same retrieved passages across the whole generated 
sequence, the other can use different passages per token. We fine-tune and evaluate our models on a wide range of 
knowledge-intensive NLP tasks and set the state-of-the-art on three open domain QA tasks, outperforming parametric 
seq2seq models and task-specific retrieve-and-extract architectures. For language generation tasks, we find th

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Where do Ice Bears live?\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n[Quote from Conversation] Agent previously responded with  Universal RAG is 
designed to retrieve and integrate knowledge from heterogeneous sources with diverse modalities and granularities. 
This means that it can work with various types of data, not just text. According to the document, Universal RAG 
uses modality-aware routing, which dynamically identifies the most appropriate modality-specific corpus and 
performs targeted retrieval within it.\n\nSources:\n\n* UniversalRAG: Retrieval-Augmented Generation over Corpora 
of Diverse Modalities and Granularities (arxiv.org/abs/2504.20734v3)\n* Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks (arxiv.org/abs/2005.10439)\n[Quote from Conversation] User previously responded with 
What Modalities does Universal RAG make use of?\n[Quote from Conversation] User previously responded with Tell me 
about RAG!\n[Quote from Conversation] Agent previously responded with  RAG, or Retrieval-Augmented Generation, is a
concept in the field of natural language processing (NLP) that involves using both parametric and non-parametric 
memory to improve the performance of language generation models on knowledge-intensive tasks.\n\nParametric memory 
refers to the knowledge that a model learns during training and stores in its parameters. Non-parametric memory, on
the other hand, is external knowledge that a model can retrieve as needed. In the case of RAG, this non-parametric 
memory is accessed through a dense vector index of a large corpus, such as Wikipedia, using a pre-trained neural 
retriever.\n\nThere are different formulations of RAG, such as one where the same retrieved passages are used 
across the entire generated sequence, and another where different passages can be used for each token. RAG models 
have been shown to outperform parametric-only seq2seq models and task-specific retrieve-and-extract architectures 
on a range of knowledge-intensive NLP tasks.\n\nIn addition to the original RAG model, there is also a variant 
called UniversalRAG, which is designed to retrieve and integrate knowledge from heterogeneous sources with diverse 
modalities and granularities. This is in contrast to most existing approaches, which are limited to a text-only 
corpus. UniversalRAG includes a modality-aware routing mechanism, which dynamically identifies the most appropriate
modality-specific corpus and performs targeted retrieval within it. It has been shown to outperform various 
modality-specific and unified baselines on 10 benchmarks of multiple modalities.\n\nOverall, RAG and UniversalRAG 
offer a promising approach to improving the factual accuracy and performance of language generation models on 
knowledge-intensive tasks by allowing them to access and precisely manipulate external knowledge as needed.\n\n\n 
Document Retrieval:\n[Quote from Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena] . Hence, LLM-as-a-judge is
a scalable and explainable way to approximate human preferences, which are otherwise very expensive to obtain. 
Additionally, we show our benchmark and traditional benchmarks complement each other by evaluating several variants
of LLaMA and Vicuna. The MT-bench questions, 3K expert votes, and 30K conversations with human preferences are 
publicly available at https://github.com/lm-sys/FastChat/tree/main/fastchat/llm_judge.\n[Quote from Attention Is 
All You Need] .5 days on eight GPUs, a small fraction of the training costs of the best models from the literature.
We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency 
parsing both with large and limited training data.\n[Quote from FFPDG: Fast,

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://094eaf58be01da97d1.gradio.live
Closing server running on port: 7860


<br>

----

<br>

## **Part 4:** Saving Your Index For Evaluation

After you've implemented your RAG chain, please save your accumulated vector store as shown [in the official documentation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading). You'll have a chance to use it again for your final assessment!

In [ ]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed). After you have confirmed that the cell can pull in your index, download `docstore_index.tgz` for use in the last notebook!

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

-----

<br>

## **Part 5:** Wrap-Up

Congratulations! Assuming your RAG chain is all good, you're now ready to move on to the **RAG Evaluation [Assessment]** section!

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>